In [1]:
using Oceananigans
using Printf
using GLMakie

using Random, Statistics

In [2]:
L = 10
H = L
grid = RectilinearGrid(size=(256, 256), x=(-L/2, L/2), z=(-H/2, H/2),
                       topology=(Periodic, Flat, Bounded))

shear_flow(x, z, t) = tanh(z)
stratification(x, z, t, p) = p.h * p.Ri * tanh((z + 0.2*sin(2π*x/L)) / p.h)

U = BackgroundField(shear_flow)
B = BackgroundField(stratification, parameters=(Ri=0.1, h=1/4));

zF = znodes(grid, Face())
zC = znodes(grid, Center())

Ri, h = B.parameters

# The model
model = NonhydrostaticModel(timestepper = :RungeKutta3,
                              advection = UpwindBiased(),
                                   grid = grid,
                               coriolis = nothing,
                      background_fields = (u=U, b=B),
                                closure = ScalarDiffusivity(ν=0.5e-4, κ=0.5e-4),
                               buoyancy = BuoyancyTracer(),
                                tracers = :b)

Δt = 0.025
simulation = Simulation(model, Δt=Δt, stop_iteration=40000, verbose=false)

Simulation of NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 25 ms
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: Inf days
├── Stop iteration: 40000.0
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   └── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [3]:
u, v, w = model.velocities
b = model.tracers.b
xb, yb, zb = nodes(b)
total_b = Field(b + model.background_fields.tracers.b)

mean_perturbation_kinetic_energy = Field(Average(1/2 * (u^2 + w^2)))

noise(x, z) = 1.e-3*randn()
bᵢ(x, z) = noise(x, z)

set!(model, u=noise, w=noise, b=bᵢ)

In [4]:
simulation.output_writers[:buoyancy] =
    JLD2Writer(model, (b=b, B=total_b),
                     schedule = TimeInterval(0.5),
                     filename = "../data/raw_simulation_output/stratified_shear/stratified_instability.jld2",
                     overwrite_existing = true)

JLD2Writer scheduled on TimeInterval(500 ms):
├── filepath: ../data/raw_simulation_output/stratified_shear/stratified_instability.jld2
├── 2 outputs: (b, B)
├── array type: Array{Float32}
├── including: [:grid, :coriolis, :buoyancy, :closure]
├── file_splitting: NoFileSplitting
└── file size: 34.7 KiB

In [5]:
@info "*** Running a simulation of Kelvin-Helmholtz instability..."
run!(simulation)

[ Info: *** Running a simulation of Kelvin-Helmholtz instability...
┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/phaon/src/JLD2.jl:153


LoadError: SystemError: opening file "/Users/henrifdrake/code/ESS130/data/raw_simulation_output/stratified_shear/stratified_instability.jld2": No such file or directory

In [6]:
@info "Making a neat movie of stratified shear flow..."

filepath = simulation.output_writers[:buoyancy].filepath

B_timeseries = FieldTimeSeries(filepath, "B")

times = B_timeseries.times
t_final = times[end]

n = Observable(1)

Bₙ = @lift interior(B_timeseries, :, 1, :, $n)

fig = Figure(size=(800, 600))

kwargs = (xlabel="x [m]", ylabel="z [m]", limits = ((xb[1], xb[end]), (-3, 3)), aspect=1,)

title = @lift @sprintf("buoyancy [m/s²] at t = %.2f", times[$n])

ax_B = Axis(fig[1, 1]; title = title, kwargs...)

B_lims = (-maximum(abs, interior(B_timeseries, :, 1, :, :)), maximum(abs, interior(B_timeseries, :, 1, :, :)))

hm_B = heatmap!(ax_B, xb, zb, Bₙ; colorrange = B_lims, colormap = :balance)
Colorbar(fig[1, 2], hm_B)

frames = 1:length(times)

record(fig, "../movies/Lecture19_shear_instability.mp4", frames, framerate=8) do i
    @info "Plotting frame $i of $(frames[end])..."
    n[] = i
end

[ Info: Making a neat movie of stratified shear flow...


LoadError: Glob pattern cannot be empty or start with a / character